# Train VAE

## Load data

In [ ]:
# prerequisites
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image
from torch.utils.data import Dataset, DataLoader
from VAE import *

bs = 100
# MNIST Dataset
class MNISTVideoDataset(Dataset):
    def __init__(self, videos):
        self.videos = videos

    def __len__(self):
        return len(self.videos)

    def __getitem__(self, idx):
        video = self.videos[idx]
        # Convert video frames to PyTorch tensors
        video_tensor = torch.tensor(video, dtype=torch.float32)
        return video_tensor

# Create an instance of the custom dataset
train_dataset = MNISTVideoDataset(np.load("./image_data/vae_train.npy"))
test_dataset = MNISTVideoDataset(np.load("./image_data/vae_test.npy"))

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=bs, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=bs, shuffle=False)


## Build model

In [ ]:
# build model
vae = VAE(x_dim=64*64, h_dim1= 256, h_dim2=64, z_dim=2)
if torch.cuda.is_available():
    vae.cuda()
optimizer = optim.Adam(vae.parameters(),lr=1e-3)
# return reconstruction error + KL divergence losses
def loss_function(recon_x, x, mu, log_var):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 64*64), reduction='sum')
    KLD = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    return BCE + KLD

def train(epoch,t=None):
    vae.train()
    train_loss = 0
    for batch_idx, data in enumerate(train_loader):
        data = data.cuda()
        optimizer.zero_grad()
        
        recon_batch, mu, log_var = vae(data)
        loss = loss_function(recon_batch, data, mu, log_var)
        
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            if t is not None:
                t.set_description('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item() / len(data)))
    if t is not None:
        t.set_description('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))
def test(t=None):
    vae.eval()
    test_loss= 0
    with torch.no_grad():
        for data in test_loader:
            data = data.cuda()
            recon, mu, log_var = vae(data)
            
            # sum up batch loss
            test_loss += loss_function(recon, data, mu, log_var).item()
        
    test_loss /= len(test_loader.dataset)
    if t is not None:
        t.set_description('====> Test set loss: {:.4f}'.format(test_loss))


## Start training

In [ ]:
from tqdm import trange
num_epoches=500

vae_filename="pretrained_vae.pt"
with trange(num_epoches) as t:
    for epoch in t:
        train(epoch,t)
        test(t)
        with torch.no_grad():
            z = torch.randn(64,2 ).cuda()
            sample = vae.decoder(z).cuda()
            save_image(sample.view(64, 1, 64, 64), f"./samples/epoch_{epoch}_sample_" + '.png')
torch.save(vae,vae_filename)

In [ ]:
from scipy.stats import norm
import matplotlib.pyplot as plt

# Display a 2D manifold of the digits
n = 20  # figure with 20x20 digits
digit_size = 64
figure = torch.zeros((digit_size * n, digit_size * n))

# Construct grid of latent variable values
grid_x = norm.ppf(np.linspace(0.05, 0.95, n))
grid_y = norm.ppf(np.linspace(0.05, 0.95, n))

# decode for each square in the grid
for i, yi in enumerate(grid_x):
    for j, xi in enumerate(grid_y):
        Z_sample = np.array([[xi, yi]])
        Z_sample = torch.tensor(Z_sample,dtype=torch.float32)
        Z_sample=Z_sample.cuda()
        #print(Z_sample)
        
        X_decoded = vae.decoder(Z_sample)
        digit = X_decoded[0].reshape(digit_size, digit_size)
        figure[i * digit_size: (i + 1) * digit_size,
               j * digit_size: (j + 1) * digit_size] =digit

plt.figure(figsize=(10, 10))
plt.imshow(figure.detach().numpy(), cmap='gnuplot2')
plt.show()  

In [ ]:
from scipy.stats import norm
import matplotlib.pyplot as plt

# Display a 2D manifold of the digits
n = 20  # figure with 20x20 digits
digit_size = 64
figure = torch.zeros((digit_size * n, digit_size * n))

# Construct grid of latent variable values
grid_x = norm.ppf(np.linspace(0.05, 0.95, n))
grid_y = norm.ppf(np.linspace(0.05, 0.95, n))

# decode for each square in the grid
for i, yi in enumerate(grid_x):
    for j, xi in enumerate(grid_y):
        Z_sample = np.array([[xi, yi]])
        Z_sample = torch.tensor(Z_sample,dtype=torch.float32)
        Z_sample=Z_sample.cuda()
        #print(Z_sample)
        
        X_decoded = vae.decoder(Z_sample)
        digit = X_decoded[0].reshape(digit_size, digit_size)
        figure[i * digit_size: (i + 1) * digit_size,
               j * digit_size: (j + 1) * digit_size] =digit

plt.figure(figsize=(10, 10))
plt.imshow(figure.detach().numpy(), cmap='gnuplot2')
plt.show()  